# Commitment Schemes: привязка и сокрытие

В этом ноутбуке мы:
1. Реализуем hash-based commitment (SHA-256)
2. Проверим свойства binding и hiding
3. Реализуем Pedersen commitment на эллиптической кривой
4. Продемонстрируем гомоморфное свойство
5. Сравним оба подхода

**Предварительные знания:** SHA-256 (CRYPTO-03/04), эллиптические кривые (CRYPTO-09), точки на кривой (CRYPTO-10)

## 1. Hash-based Commitment

In [ ]:
import hashlib
import os

def hash_commit(message: bytes) -> tuple:
    """
    Создает hash-based commitment.
    C = SHA-256(message || r), где r -- случайный blinding factor (32 байта).
    Возвращает (commitment_hex, blinding_factor).
    """
    r = os.urandom(32)  # Случайный blinding factor
    commitment = hashlib.sha256(message + r).hexdigest()
    return commitment, r


def hash_verify(commitment: str, message: bytes, r: bytes) -> bool:
    """
    Проверяет opening commitment.
    Возвращает True если H(message || r) == commitment.
    """
    return hashlib.sha256(message + r).hexdigest() == commitment


# Демонстрация: Алиса коммитит свой голос "ДА"
vote = b"DA"  # Голос Алисы
C, r = hash_commit(vote)

print("=== Фаза COMMIT ===")
print(f"Сообщение: {vote}")
print(f"Blinding factor r: {r.hex()[:32]}...")
print(f"Commitment C: {C}")
print()

# Фаза REVEAL: Алиса раскрывает (vote, r)
print("=== Фаза REVEAL ===")
valid = hash_verify(C, vote, r)
print(f"Проверка H('{vote.decode()}' || r) == C: {valid}")

# Попытка подменить голос
fake_vote = b"NET"
fake_valid = hash_verify(C, fake_vote, r)
print(f"Проверка H('NET' || r) == C: {fake_valid}  <-- binding: подмена невозможна!")

## 2. Свойство Binding

**Binding** означает: после commit Алиса НЕ МОЖЕТ найти другое сообщение m' и r', такие что H(m' || r') == H(m || r).
Это потребовало бы найти коллизию SHA-256 -- вычислительно невозможная задача.

In [ ]:
import time

# Попытка нарушить binding: найти m' != m с таким же commitment
target_commitment = C  # Commitment Алисы
original_msg = vote

print("Попытка найти коллизию (нарушить binding)...")
print(f"Target: {target_commitment[:16]}...")
print()

attempts = 100_000
start = time.time()
found = False

for i in range(attempts):
    # Пробуем случайные сообщения и blinding factors
    fake_msg = f"fake_{i}".encode()
    fake_r = os.urandom(32)
    fake_c = hashlib.sha256(fake_msg + fake_r).hexdigest()
    
    if fake_c == target_commitment:
        found = True
        print(f"Коллизия найдена за {i} попыток! (это было бы сенсацией)")
        break

elapsed = time.time() - start

if not found:
    print(f"Коллизия НЕ найдена за {attempts:,} попыток ({elapsed:.3f}с)")
    print(f"Для SHA-256 нужно ~2^128 попыток для коллизии")
    print(f"При {attempts/elapsed:.0f} попыток/сек это заняло бы ~{2**128 / (attempts/elapsed) / 3.15e7:.1e} лет")
    print("\nBinding: ОБЕСПЕЧЕНО (вычислительно невозможно нарушить)")

## 3. Свойство Hiding

**Hiding** означает: зная commitment C, Боб НЕ МОЖЕТ определить сообщение m.
Случайный blinding factor r делает это невозможным, даже если пространство сообщений маленькое.

In [ ]:
# Демонстрация: ПОЧЕМУ нужен blinding factor

# БЕЗ blinding factor (наивная схема)
print("=== Наивная схема: C = H(m) без blinding factor ===")
c_da = hashlib.sha256(b"DA").hexdigest()
c_net = hashlib.sha256(b"NET").hexdigest()
print(f"H('DA')  = {c_da[:16]}...")
print(f"H('NET') = {c_net[:16]}...")
print("Боб видит commitment и просто хеширует оба варианта -- hiding НАРУШЕНО!")
print()

# С blinding factor
print("=== Правильная схема: C = H(m || r) ===")
C_with_r, _ = hash_commit(b"DA")
print(f"C = H('DA' || random_r) = {C_with_r[:16]}...")
print("Боб видит C, но НЕ знает r -> не может проверить 'DA' vs 'NET'")
print()

# Попытка Боба угадать
print("=== Попытка Боба угадать (перебор) ===")
# Даже зная что m in {'DA', 'NET'}, Боб должен перебрать 2^256 значений r
print(f"Пространство blinding factors: 2^256 = ~{2**256:.1e}")
print("Перебор r для каждого кандидата m невозможен")
print("\nHiding: ОБЕСПЕЧЕНО (случайный r делает C неотличимым от случайного числа)")

## 4. Pedersen Commitment на secp256k1

Pedersen commitment использует эллиптические кривые:
- C = m*G + r*H
- G -- стандартный генератор secp256k1
- H -- второй генератор (derived, дискретный логарифм относительно G неизвестен)

In [ ]:
from ecdsa import SECP256k1, ellipticcurve
import hashlib
import random

# Параметры кривой secp256k1
curve = SECP256k1.curve
G = SECP256k1.generator
order = SECP256k1.order

print(f"Кривая: secp256k1")
print(f"Порядок группы: {order}")
print(f"G.x = {G.x()}")
print(f"G.y = {G.y()}")
print()

# Получаем второй генератор H методом "nothing-up-my-sleeve"
# H = hash_to_scalar("Pedersen_generator_H") * G
# Это гарантирует что дискретный логарифм H относительно G неизвестен
H_seed = hashlib.sha256(b"Pedersen_generator_H").digest()
H_scalar = int.from_bytes(H_seed, 'big') % order
H = H_scalar * G

print(f"Второй генератор H (derived from 'Pedersen_generator_H'):")
print(f"H.x = {H.x()}")
print(f"H.y = {H.y()}")
print(f"H_scalar = {H_scalar} (НЕ дискретный логарифм H по базе G -- это seed)")

In [ ]:
def pedersen_commit(m: int, r: int) -> ellipticcurve.PointJacobi:
    """
    Pedersen commitment: C = m*G + r*H
    m -- значение (секретное)
    r -- blinding factor (случайный)
    """
    return m * G + r * H


def pedersen_open(C, m: int, r: int) -> bool:
    """
    Проверка opening Pedersen commitment.
    Возвращает True если m*G + r*H == C.
    """
    expected = m * G + r * H
    return C.x() == expected.x() and C.y() == expected.y()


# Демонстрация: commit to age = 25
m = 25  # Возраст
r = random.randint(1, order - 1)  # Случайный blinding factor

C = pedersen_commit(m, r)

print("=== Pedersen Commitment ===")
print(f"Значение m = {m} (возраст)")
print(f"Blinding r = {r}")
print(f"C = m*G + r*H")
print(f"C.x = {C.x()}")
print(f"C.y = {C.y()}")
print()

# Verify opening
valid = pedersen_open(C, m, r)
print(f"Проверка opening (m={m}, r): {valid}")

# Попытка открыть с другим значением
fake_valid = pedersen_open(C, 30, r)
print(f"Проверка opening (m=30, r): {fake_valid}  <-- binding: подмена невозможна!")

## 5. Гомоморфное свойство Pedersen

Ключевое преимущество Pedersen: **гомоморфность**.

C(a, r1) + C(b, r2) = (a*G + r1*H) + (b*G + r2*H) = (a+b)*G + (r1+r2)*H = C(a+b, r1+r2)

Можно СКЛАДЫВАТЬ commitments без раскрытия значений!

In [ ]:
# Два значения
a = 7
b = 13
r1 = random.randint(1, order - 1)
r2 = random.randint(1, order - 1)

# Отдельные commitments
C1 = pedersen_commit(a, r1)
C2 = pedersen_commit(b, r2)

# Сумма commitments (точечное сложение на кривой)
C_sum = C1 + C2

# Commitment суммы напрямую
C_direct = pedersen_commit(a + b, (r1 + r2) % order)

print("=== Гомоморфное свойство ===")
print(f"a = {a}, r1 = {r1}")
print(f"b = {b}, r2 = {r2}")
print(f"a + b = {a + b}")
print()

print(f"C(a) = {a}*G + r1*H")
print(f"C(b) = {b}*G + r2*H")
print()

# Проверяем равенство
match = (C_sum.x() == C_direct.x()) and (C_sum.y() == C_direct.y())
print(f"C(a) + C(b) == C(a+b, r1+r2): {match}")
print()

if match:
    print("ГОМОМОРФНОСТЬ ПОДТВЕРЖДЕНА!")
    print("C(a) + C(b) = (a*G + r1*H) + (b*G + r2*H)")
    print("           = (a+b)*G + (r1+r2)*H")
    print("           = C(a+b, r1+r2)")
    print()
    print("Можно сложить commitments БЕЗ раскрытия a и b!")

## 6. Практическое применение: приватное голосование

Сценарий: 5 голосующих коммитят голоса (1 = ДА, 0 = НЕТ). Благодаря гомоморфности, можно подсчитать сумму без раскрытия отдельных голосов.

In [ ]:
# 5 голосующих
votes = [1, 0, 1, 1, 0]  # ДА, НЕТ, ДА, ДА, НЕТ
blinding_factors = [random.randint(1, order - 1) for _ in range(5)]

# Фаза COMMIT: каждый публикует commitment
print("=== Фаза COMMIT ===")
commitments = []
for i, (v, bf) in enumerate(zip(votes, blinding_factors)):
    C = pedersen_commit(v, bf)
    commitments.append(C)
    vote_label = 'ДА' if v == 1 else 'НЕТ'
    print(f"Voter {i+1}: commitment C_{i+1} опубликован (голос '{vote_label}' скрыт)")

print()

# Подсчёт суммы commitments (без раскрытия голосов!)
C_total = commitments[0]
for c in commitments[1:]:
    C_total = C_total + c

print("=== Подсчёт (гомоморфный) ===")
print(f"Сумма commitments: C_total = C_1 + C_2 + C_3 + C_4 + C_5")
print(f"C_total.x = {C_total.x()}")
print()

# Фаза REVEAL: все раскрывают голоса
print("=== Фаза REVEAL ===")
total_votes = sum(votes)
total_r = sum(blinding_factors) % order

# Проверяем: сумма commitments == commitment суммы
C_expected = pedersen_commit(total_votes, total_r)
sum_match = (C_total.x() == C_expected.x()) and (C_total.y() == C_expected.y())

print(f"Сумма голосов: {total_votes} из 5 (ДА)")
print(f"Проверка: sum(C_i) == C(sum(v_i), sum(r_i)): {sum_match}")
print()
print(f"Результат: {total_votes} ДА, {5 - total_votes} НЕТ")
if sum_match:
    print("Голосование ВЕРИФИЦИРОВАНО (никто не подменил голос)")

## 7. Сравнение Hash vs Pedersen

In [ ]:
print("Сравнение Hash-based и Pedersen Commitment")
print("=" * 60)
print(f"{'Свойство':<20} {'Hash-based':<20} {'Pedersen':<20}")
print("-" * 60)
print(f"{'Формула':<20} {'C=H(m||r)':<20} {'C=mG+rH':<20}")
print(f"{'Binding':<20} {'Computational':<20} {'Computational':<20}")
print(f"{'Hiding':<20} {'Perfect':<20} {'Perfect':<20}")
print(f"{'Гомоморфность':<20} {'НЕТ':<20} {'ДА!':<20}")
print(f"{'Setup':<20} {'Нет':<20} {'G, H генераторы':<20}")
print(f"{'Скорость':<20} {'Быстро (hash)':<20} {'Медленно (EC)':<20}")
print(f"{'Применения':<20} {'Voting, lottery':<20} {'ZK proofs, CT':<20}")
print("=" * 60)
print()
print("Ключевое различие: гомоморфность Pedersen позволяет ВЫЧИСЛЯТЬ")
print("на зашифрованных данных. Это фундамент для ZK-доказательств.")

## 8. Упражнения

### Упражнение 1: Bit Commitment для Range Proof

Реализуйте commitment к каждому биту числа x (0 <= x < 256).
Каждый бит -- отдельный Pedersen commitment.
Покажите что сумма bit-commitments (взвешенная по 2^i) равна commitment к x.

In [ ]:
# Упражнение 1: Ваш код здесь
# Подсказка:
# x = b7*128 + b6*64 + ... + b1*2 + b0*1
# C(x, r) = sum(bi * 2^i * G) + r*H
# Нужно: C(b0, r0), C(b1, r1), ..., и показать что
# sum(2^i * C(bi, ri)) == C(x, sum(2^i * ri))


### Упражнение 2: Coin Flip Protocol

Реализуйте протокол "честного подбрасывания монетки" между Алисой и Бобом:
1. Алиса выбирает бит a (0 или 1), коммитит C_a
2. Боб выбирает бит b (0 или 1) открыто
3. Алиса раскрывает a
4. Результат = a XOR b (честный, так как Алиса не могла подменить a, Боб не знал a)

In [ ]:
# Упражнение 2: Ваш код здесь
# Подсказка:
# alice_bit = random.choice([0, 1])
# C, r = hash_commit(str(alice_bit).encode())
# bob_bit = random.choice([0, 1])
# alice раскрывает (alice_bit, r)
# проверяем commitment, вычисляем результат
